## Description of Project

In [1]:
%reload_ext autoreload
%autoreload 2

# Set path
import os
import sys
sys.path.append(os.path.abspath('..'))

# Standard library imports
import ta
import pandas as pd
import numpy as np
import yfinance as yf
import pandera as pa
from loguru import logger
    
# Custom library imports
from src.utils.logger_utils import setup_logger

### Setup

In [2]:
# Setup logger
setup_logger()

2024-07-20 19:21:06.851 | DEBUG    | src.utils.logger_utils:setup_logger:33 - Setting up logger...


### Fetch Data

In [3]:
def fetch_stock_data(tickers: list, start: str, end: str, interval: str) -> pd.DataFrame:
    """
    Fetch stock data from Yahoo Finance.
    
    Parameters:
        tickers (list): List of stock ticker symbols.
        start (str): Start date in 'YYYY-MM-DD' format.
        end (str): End date in 'YYYY-MM-DD' format.
        interval (str): Data interval (e.g., '1d' for daily, '1m' for minute-level, etc.).
        
    Returns:
        pd.DataFrame: Stock data.
    """
    logger.info(f'Fetching data for {tickers} from {start} to {end} in {interval} intervals...')
        
    stock_df = yf.download(tickers, start=start, end=end, interval=interval)            
    
    return stock_df

### Pre-process Data

In [4]:
def pre_process_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Clean and validate stock data.

    Parameters:
        df (pd.DataFrame): The DataFrame containing stock data with columns including 'Open', 'High',
                           'Low', 'Close', 'Adj Close', and 'Volume'.

    Returns:
        pd.DataFrame: The cleaned and validated DataFrame.
    """
    logger.info(f'Pre-processing data...')
    
    # Define the schema
    schema = pa.DataFrameSchema({
        "Open": pa.Column(pa.Float, nullable=False),
        "High": pa.Column(pa.Float, nullable=False),
        "Low": pa.Column(pa.Float, nullable=False),
        "Close": pa.Column(pa.Float, nullable=False),
        "Adj Close": pa.Column(pa.Float, nullable=False),
        "Volume": pa.Column(pa.Int, nullable=False)
    })

    # Remove rows with missing values
    df = df.dropna()
    
    # Reset index to have a clean DataFrame
    df.reset_index(drop=True)
    
    # Check for and remove duplicate entries
    df = df[~df.index.duplicated(keep='first')]
    
    # Validate the schema
    validated_df = schema.validate(df)
    
    return validated_df

### Calculating Statistical Metrics

In [5]:
def calculate_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate statistical metrics for stock data.

    Parameters:
        df (pd.DataFrame): The DataFrame containing stock data.

    Returns:
        pd.DataFrame: The DataFrame with statistical metrics.
    """
    logger.info(f'Calculating statistical metrics...')
    
     # Simple Moving Averages
    df['SMA_20'] = ta.trend.SMAIndicator(df['Close'], window=20).sma_indicator()
    df['SMA_50'] = ta.trend.SMAIndicator(df['Close'], window=50).sma_indicator()
    
    # Exponential Moving Averages
    df['EMA_12'] = ta.trend.ema_indicator(df['Close'], window=12)
    df['EMA_26'] = ta.trend.ema_indicator(df['Close'], window=26)

    # Bollinger Bands
    bollinger = ta.volatility.BollingerBands(close=df['Close'], window=20, window_dev=2)
    df['BB_Middle'] = bollinger.bollinger_mavg()
    df['BB_Upper'] = bollinger.bollinger_hband()
    df['BB_Lower'] = bollinger.bollinger_lband()

    # RSI (Relative Strength Index)
    df['RSI'] = ta.momentum.RSIIndicator(close=df['Close'], window=14).rsi()

    # MACD (Moving Average Convergence Divergence)
    macd = ta.trend.MACD(close=df['Close'])
    df['MACD'] = macd.macd()
    df['MACD_Signal'] = macd.macd_signal()

    # ATR (Average True Range)
    df['ATR'] = ta.volatility.AverageTrueRange(high=df['High'], low=df['Low'], close=df['Close'], window=14).average_true_range()
    
    return df

### Implement a Trading Strategy

In [6]:
def define_trading_strategy(df: pd.DataFrame) -> pd.DataFrame:
    """
    Define a trading strategy based on statistical indicators.

    Parameters:
        df (pd.DataFrame): DataFrame containing stock data with statistical metrics.

    Returns:
        pd.DataFrame: DataFrame with trading strategy signals.
    """
    logger.info(f'Defining trading strategy...')   

    # SMA Crossover Strategy
    df['Buy_Signal_SMA'] = (df['SMA_20'] > df['SMA_50']) & (df['SMA_20'].shift(1) <= df['SMA_50'].shift(1))
    df['Sell_Signal_SMA'] = (df['SMA_20'] < df['SMA_50']) & (df['SMA_20'].shift(1) >= df['SMA_50'].shift(1))
    
   # EMA Crossover Strategy
    df['Buy_Signal_EMA'] = (df['EMA_12'] > df['EMA_26']) & (df['EMA_12'].shift(1) <= df['EMA_26'].shift(1))
    df['Sell_Signal_EMA'] = (df['EMA_12'] < df['EMA_26']) & (df['EMA_12'].shift(1) >= df['EMA_26'].shift(1))

    # Bollinger Reversal Strategy
    df['Buy_Signal_BB'] = (df['Close'] <= df['BB_Lower'])
    df['Sell_Signal_BB'] = (df['Close'] >= df['BB_Upper'])

    # RSI Momentum Strategy
    df['Buy_Signal_RSI'] = (df['RSI'] > 30) & (df['RSI'].shift(1) <= 30)
    df['Sell_Signal_RSI'] = (df['RSI'] < 70) & (df['RSI'].shift(1) >= 70)

    # MACD Signal Line Strategy
    df['Buy_Signal_MACD'] = (df['MACD'] > df['MACD_Signal']) & (df['MACD'].shift(1) <= df['MACD_Signal'].shift(1))
    df['Sell_Signal_MACD'] = (df['MACD'] < df['MACD_Signal']) & (df['MACD'].shift(1) >= df['MACD_Signal'].shift(1))

    # Set stop-loss and take-profit based on ATR
    df['Stop_Loss'] = df['Close'] - (df['ATR'] * 1.5)
    df['Take_Profit'] = df['Close'] + (df['ATR'] * 1.5)

    return df

### Backtesting

In [7]:
# def fetch_stock_data(tickers: list, start: str, end: str, interval: str) -> pd.DataFrame:
#     stock_df = yf.download(tickers, start=start, end=end, interval=interval)
#     return stock_df

# def pre_process_data(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.dropna()
#     df = df.reset_index()
#     df.index = pd.to_datetime(df['Date'])
#     df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
#     return df

# def calculate_indicators(df: pd.DataFrame) -> pd.DataFrame:
#     df['SMA_20'] = df['Close'].rolling(window=20).mean()
#     df['SMA_50'] = df['Close'].rolling(window=50).mean()
#     df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
#     df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
#     df['BB_Middle'] = df['Close'].rolling(window=20).mean()
#     df['BB_Upper'] = df['BB_Middle'] + (df['Close'].rolling(window=20).std() * 2)
#     df['BB_Lower'] = df['BB_Middle'] - (df['Close'].rolling(window=20).std() * 2)
#     df['RSI'] = 100 - (100 / (1 + (df['Close'].diff().clip(lower=0).rolling(window=14).mean() / df['Close'].diff().clip(upper=0).abs().rolling(window=14).mean())))
#     df['MACD'] = df['EMA_12'] - df['EMA_26']
#     df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
#     df['ATR'] = df[['High', 'Low']].apply(lambda x: x['High'] - x['Low'], axis=1).rolling(window=14).mean()
#     return df

# def backtest_strategy(df: pd.DataFrame, strategy: str, initial_cash: float, stake_amount: float, commission: float) -> dict:
#     df = calculate_indicators(df)
#     df = df.dropna()

#     cash = initial_cash
#     position = 0
#     trades = []
#     trade_active = False

#     for i in range(len(df)):
#         row = df.iloc[i]

#         if strategy == 'SMA':
#             buy_signal = row['SMA_20'] > row['SMA_50']
#             sell_signal = row['SMA_20'] < row['SMA_50']
#         elif strategy == 'EMA':
#             buy_signal = row['EMA_12'] > row['EMA_26']
#             sell_signal = row['EMA_12'] < row['EMA_26']
#         elif strategy == 'BB':
#             buy_signal = row['Close'] <= row['BB_Lower']
#             sell_signal = row['Close'] >= row['BB_Upper']
#         elif strategy == 'RSI':
#             buy_signal = row['RSI'] > 30
#             sell_signal = row['RSI'] < 70
#         elif strategy == 'MACD':
#             buy_signal = row['MACD'] > row['MACD_Signal']
#             sell_signal = row['MACD'] < row['MACD_Signal']
#         else:
#             raise ValueError("Unknown strategy")

#         if buy_signal and not trade_active:
#             if cash >= stake_amount:
#                 shares_bought = stake_amount / row['Close']
#                 cash -= stake_amount + commission
#                 position += shares_bought
#                 trades.append((row.name, 'BUY', row['Close'], shares_bought))
#                 trade_active = True

#         elif sell_signal and trade_active:
#             cash += position * row['Close'] - commission
#             trades.append((row.name, 'SELL', row['Close'], position))
#             position = 0
#             trade_active = False

#     cash += position * df.iloc[-1]['Close'] - commission if position > 0 else cash
#     final_value = cash + (position * df.iloc[-1]['Close'])

#     return {
#         'Final Cash': cash,
#         'Final Position Value': position * df.iloc[-1]['Close'],
#         'Final Total Value': final_value,
#         'Trades': trades
#     }

# def run_backtest(start_date, end_date, initial_cash, strategy):
#     df = fetch_stock_data(['GOOGL'], start_date, end_date, '1d')
#     df = pre_process_data(df)

#     stake_amount = 0.8
#     commission = 2
#     results = backtest_strategy(df, strategy, initial_cash, stake_amount, commission)
    
#     return results

# # Define your test period
# start_date = '2023-07-18'
# end_date = '2024-07-20'
# initial_cash = 100  # Initial cash in EUR

# # Run the backtests for all strategies
# strategies = ['SMA', 'EMA', 'BB', 'RSI', 'MACD']
# results = {}

# for strategy in strategies:
#     print(f"Running backtest for {strategy} strategy...")
#     result = run_backtest(start_date, end_date, initial_cash, strategy)
#     results[strategy] = result

# # Print results
# for strategy, result in results.items():
#     print(f"\nStrategy: {strategy}")
#     print(f"Final Cash: {result['Final Cash']}")
#     print(f"Final Position Value: {result['Final Position Value']}")
#     print(f"Final Total Value: {result['Final Total Value']}")
#     print(f"Number of Trades: {len(result['Trades'])}")
#     for trade in result['Trades']:
#         print(f"Trade Date: {trade[0]}, Type: {trade[1]}, Price: {trade[2]}, Shares: {trade[3]}")


In [8]:
# def fetch_stock_data(tickers: list, start: str, end: str, interval: str) -> pd.DataFrame:
#     stock_df = yf.download(tickers, start=start, end=end, interval=interval)
#     return stock_df

# def pre_process_data(df: pd.DataFrame) -> pd.DataFrame:
#     df = df.dropna()
#     df = df.reset_index()
#     df.index = pd.to_datetime(df['Datetime'])
#     df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
#     return df

# def calculate_indicators(df: pd.DataFrame) -> pd.DataFrame:
#     df['SMA_20'] = df['Close'].rolling(window=20).mean()
#     df['SMA_50'] = df['Close'].rolling(window=50).mean()
#     df['EMA_12'] = df['Close'].ewm(span=12, adjust=False).mean()
#     df['EMA_26'] = df['Close'].ewm(span=26, adjust=False).mean()
#     df['BB_Middle'] = df['Close'].rolling(window=20).mean()
#     df['BB_Upper'] = df['BB_Middle'] + (df['Close'].rolling(window=20).std() * 2)
#     df['BB_Lower'] = df['BB_Middle'] - (df['Close'].rolling(window=20).std() * 2)
#     df['RSI'] = 100 - (100 / (1 + (df['Close'].diff().clip(lower=0).rolling(window=14).mean() / df['Close'].diff().clip(upper=0).abs().rolling(window=14).mean())))
#     df['MACD'] = df['EMA_12'] - df['EMA_26']
#     df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
#     df['ATR'] = df[['High', 'Low']].apply(lambda x: x['High'] - x['Low'], axis=1).rolling(window=14).mean()
#     return df

# def calculate_position_size(cash: float, volatility: float, stake_percentage: float) -> float:
#     # Dynamic position sizing based on volatility and stake percentage
#     return cash * stake_percentage / volatility

# def backtest_strategy(df: pd.DataFrame, strategy: str, initial_cash: float, stake_percentage: float, commission: float, stop_loss_pct: float, take_profit_pct: float) -> dict:
#     df = calculate_indicators(df)
#     df = df.dropna()

#     cash = initial_cash
#     position = 0
#     trades = []
#     trade_active = False
#     entry_price = 0
#     stop_loss = 0
#     take_profit = 0

#     for i in range(len(df)):
#         row = df.iloc[i]
#         volatility = df['ATR'].iloc[i] if 'ATR' in df.columns else 1
#         stake_amount = calculate_position_size(cash, volatility, stake_percentage)

#         if strategy == 'SMA':
#             buy_signal = row['SMA_20'] > row['SMA_50']
#             sell_signal = row['SMA_20'] < row['SMA_50']
#         elif strategy == 'EMA':
#             buy_signal = row['EMA_12'] > row['EMA_26']
#             sell_signal = row['EMA_12'] < row['EMA_26']
#         elif strategy == 'BB':
#             buy_signal = row['Close'] <= row['BB_Lower']
#             sell_signal = row['Close'] >= row['BB_Upper']
#         elif strategy == 'RSI':
#             buy_signal = row['RSI'] > 30
#             sell_signal = row['RSI'] < 70
#         elif strategy == 'MACD':
#             buy_signal = row['MACD'] > row['MACD_Signal']
#             sell_signal = row['MACD'] < row['MACD_Signal']
#         else:
#             raise ValueError("Unknown strategy")

#         if buy_signal and not trade_active:
#             if cash >= stake_amount:
#                 shares_bought = stake_amount / row['Close']
#                 cash -= stake_amount + commission
#                 position += shares_bought
#                 entry_price = row['Close']
#                 stop_loss = entry_price * (1 - stop_loss_pct)
#                 take_profit = entry_price * (1 + take_profit_pct)
#                 trades.append((row.name, 'BUY', row['Close'], shares_bought))
#                 trade_active = True

#         elif trade_active:
#             if row['Close'] <= stop_loss or row['Close'] >= take_profit or sell_signal:
#                 cash += position * row['Close'] - commission
#                 trades.append((row.name, 'SELL', row['Close'], position))
#                 position = 0
#                 trade_active = False

#     if position > 0:
#         cash += position * df.iloc[-1]['Close'] - commission

#     final_value = cash + (position * df.iloc[-1]['Close'])
#     return {
#         'Final Cash': cash,
#         'Final Position Value': position * df.iloc[-1]['Close'],
#         'Final Total Value': final_value,
#         'Trades': trades,
#         'Number of Trades': len(trades)
#     }

# def run_backtest(start_date, end_date, initial_cash, strategy):
#     df = fetch_stock_data(['NVDA'], start_date, end_date, '30m')
#     df = pre_process_data(df)

#     stake_percentage = 100  # Percentage of cash allocated per trade
#     commission = 2
#     stop_loss_pct = 0.03  # 3% stop loss
#     take_profit_pct = 0.06  # 6% take profit
#     results = backtest_strategy(df, strategy, initial_cash, stake_percentage, commission, stop_loss_pct, take_profit_pct)
    
#     return results

# # Define your test period
# start_date = '2024-07-01'
# end_date = '2024-07-20'
# initial_cash = 100  # Initial cash in EUR

# # Run the backtests for all strategies
# strategies = ['SMA', 'EMA', 'BB', 'RSI', 'MACD']
# results = {}

# for strategy in strategies:
#     print(f"Running backtest for {strategy} strategy...")
#     result = run_backtest(start_date, end_date, initial_cash, strategy)
#     results[strategy] = result

# # Print results
# for strategy, result in results.items():
#     print(f"\nStrategy: {strategy}")
#     print(f"Final Cash: {result['Final Cash']}")
#     print(f"Final Position Value: {result['Final Position Value']}")
#     print(f"Final Total Value: {result['Final Total Value']}")
#     print(f"Number of Trades: {result['Number of Trades']}")
#     for trade in result['Trades']:
#         print(f"Trade Date: {trade[0]}, Type: {trade[1]}, Price: {trade[2]}, Shares: {trade[3]}")


In [9]:
def custom_backtest(df: pd.DataFrame, initial_cash: float, stake_amount: float, commission: float) -> dict:
    """
    Custom backtesting function.

    Parameters:
    - df (pd.DataFrame): DataFrame with stock data, indicators, and signals.
    - initial_cash (float): Initial amount of cash for backtesting.
    - stake_amount (float): Amount to invest per trade.
    - commission (float): Commission fee per trade.

    Returns:
    - dict: A dictionary containing final cash, position value, total value, trades, and number of trades.
    """
    cash = initial_cash
    position = 0
    trades = []
    trade_active = False
    entry_price = 0
    stop_loss = 0
    take_profit = 0

    for i in range(len(df)):
        row = df.iloc[i]

        # Check if we should buy
        if (row['Buy_Signal_EMA'] or row['Buy_Signal_BB'] or row['Buy_Signal_RSI'] or row['Buy_Signal_MACD']) and not trade_active:
            if cash >= stake_amount:
                shares_bought = stake_amount / row['Close']
                cash -= stake_amount + commission
                position += shares_bought
                entry_price = row['Close']
                stop_loss = entry_price * (1 - row['Stop_Loss'])
                take_profit = entry_price * (1 + row['Take_Profit'])
                trades.append((row.name, 'BUY', row['Close'], shares_bought))
                trade_active = True

        # Check if we should sell
        elif trade_active:
            if (row['Sell_Signal_EMA'] or row['Sell_Signal_BB'] or row['Sell_Signal_RSI'] or row['Sell_Signal_MACD'] or
                row['Close'] <= stop_loss or row['Close'] >= take_profit):
                cash += position * row['Close'] - commission
                trades.append((row.name, 'SELL', row['Close'], position))
                position = 0
                trade_active = False

    # Final cash value including remaining position
    if position > 0:
        cash += position * df.iloc[-1]['Close'] - commission

    final_value = cash + (position * df.iloc[-1]['Close'])

    return {
        'Final Cash': cash,
        'Final Position Value': position * df.iloc[-1]['Close'],
        'Final Total Value': final_value,
        'Trades': trades,
        'Number of Trades': len(trades)
    }

### Run Stock Trading

In [10]:
start_date = '2023-07-18'
end_date = '2024-07-20'
interval = '1d'
tickers = ['GOOGL']
df = fetch_stock_data(tickers, start_date, end_date, interval)
# display(df)

df = pre_process_data(df)
# display(df)

df = calculate_metrics(df)
# display(df)

df = define_trading_strategy(df)
display(df)

initial_cash = 100
stake_amount = 0.8
commission = 2
results = custom_backtest(df, initial_cash, stake_amount, commission)
print(results)


2024-07-20 19:21:07.213 | INFO     | __main__:fetch_stock_data:14 - Fetching data for ['GOOGL'] from 2023-07-18 to 2024-07-20 in 1d intervals...


[*********************100%%**********************]  1 of 1 completed

2024-07-20 19:21:07.534 | INFO     | __main__:pre_process_data:12 - Pre-processing data...


2024-07-20 19:21:08.270 | INFO     | __main__:calculate_metrics:11 - Calculating statistical metrics...
2024-07-20 19:21:08.292 | INFO     | __main__:define_trading_strategy:11 - Defining trading strategy...


,Open,High,Low,Close,Adj Close,Volume,SMA_20,SMA_50,EMA_12,EMA_26,...,Buy_Signal_EMA,Sell_Signal_EMA,Buy_Signal_BB,Sell_Signal_BB,Buy_Signal_RSI,Sell_Signal_RSI,Buy_Signal_MACD,Sell_Signal_MACD,Stop_Loss,Take_Profit
Date,,,,,,,,,,,,,,,,,,,,,
2023-07-18,124.599998,124.680000,122.959999,123.760002,123.618126,26226400,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,123.760002,123.760002
2023-07-19,124.599998,125.180000,121.800003,122.029999,121.890106,37224000,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,122.029999,122.029999
2023-07-20,121.419998,124.089996,118.220001,119.199997,119.063347,37906800,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,119.199997,119.199997
2023-07-21,120.620003,120.989998,118.730003,120.019997,119.882408,72937900,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,120.019997,120.019997
2023-07-24,121.660004,123.000000,120.980003,121.529999,121.390678,29686100,NaN,NaN,NaN,NaN,...,False,False,False,False,False,False,False,False,121.529999,121.529999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-15,184.919998,188.240005,184.919998,186.529999,186.529999,16474000,183.537502,177.3784,185.856902,182.464767,...,False,False,False,False,False,False,False,False,181.525565,191.534433
2024-07-16,187.360001,188.679993,183.369995,183.919998,183.919998,18290700,183.894002,177.7244,185.558917,182.572562,...,False,False,False,False,False,False,False,False,178.704096,189.135901
2024-07-17,182.970001,183.550003,179.899994,181.020004,181.020004,20734100,184.083002,178.0000,184.860623,182.457558,...,False,False,False,False,False,False,False,False,175.745951,186.294057


{'Final Cash': 52.124848642893355, 'Final Position Value': 0.0, 'Final Total Value': 52.124848642893355, 'Trades': [(Timestamp('2023-09-26 00:00:00'), 'BUY', 128.57000732421875, 0.006222291004329), (Timestamp('2023-09-27 00:00:00'), 'SELL', 130.5399932861328, 0.006222291004329), (Timestamp('2023-10-04 00:00:00'), 'BUY', 135.24000549316406, 0.005915409401846241), (Timestamp('2023-10-20 00:00:00'), 'SELL', 135.60000610351562, 0.005915409401846241), (Timestamp('2023-10-25 00:00:00'), 'BUY', 125.61000061035156, 0.006368919641053419), (Timestamp('2023-10-26 00:00:00'), 'SELL', 122.27999877929688, 0.006368919641053419), (Timestamp('2023-10-27 00:00:00'), 'BUY', 122.16999816894531, 0.0065482525332750145), (Timestamp('2023-12-01 00:00:00'), 'SELL', 131.86000061035156, 0.0065482525332750145), (Timestamp('2023-12-07 00:00:00'), 'BUY', 136.92999267578125, 0.005842401539407193), (Timestamp('2023-12-13 00:00:00'), 'SELL', 132.57000732421875, 0.005842401539407193), (Timestamp('2023-12-18 00:00:00'),